## Approach 3: Higher-Ranked Trait Bounds (HRTBs) and Continuation-Passing Style

In [3]:
// Approach 3: Higher-Ranked Trait Bounds (HRTBs) and Continuation-Passing Style

use std::marker::PhantomData;
use std::net::TcpStream;
use std::io::{Read, Write};

### Execution context

In [4]:

// Context trait representing the execution environment
trait Context {
    fn send(&mut self, msg: &[u8]) -> std::io::Result<()>;
    fn receive(&mut self) -> std::io::Result<Vec<u8>>;
}

// Implementation for TcpStream
impl Context for TcpStream {
    fn send(&mut self, msg: &[u8]) -> std::io::Result<()> {
        self.write_all(msg)
    }
    
    fn receive(&mut self) -> std::io::Result<Vec<u8>> {
        let mut buffer = Vec::new();
        self.read_to_end(&mut buffer)?;
        Ok(buffer)
    }
}

// Protocol trait defining what a protocol can do
trait Protocol<C: Context> {
    // The type that this protocol step continues to
    type Continue;
    
    // Execute this protocol step in the given context
    fn execute(self, context: &mut C) -> Self::Continue;
}


### Recursion combinator

In [ ]:

// Recursive protocol wrapper
struct Mu<F>(PhantomData<F>);

// Implementation for recursive protocols
impl<F, C: Context, P: Protocol<C>> Protocol<C> for Mu<F>
where
    F: FnOnce(Mu<F>) -> P,
{
    type Continue = P::Continue;
    
    fn execute(self, context: &mut C) -> Self::Continue {
        // Unfold the recursive definition once
        let protocol = (F)(self);
        // Execute the unfolded protocol
        protocol.execute(context)
    }
}



Error: expected value, found type parameter `F`

### Basic protocol operations

In [ ]:

// Basic protocol operations
struct Send<T, P> {
    message: T,
    next: P,
}

struct Receive<T, P> {
    _phantom: PhantomData<T>,
    next: P,
}

struct End;

// Protocol implementations
impl<T: Into<Vec<u8>>, P: Protocol<C>, C: Context> Protocol<C> for Send<T, P> {
    type Continue = P::Continue;
    
    fn execute(self, context: &mut C) -> Self::Continue {
        // Send the message
        context.send(&self.message.into()).expect("Failed to send");
        // Continue with the next step
        self.next.execute(context)
    }
}

impl<T: From<Vec<u8>>, P: Protocol<C>, C: Context> Protocol<C> for Receive<T, P> {
    type Continue = P::Continue;
    
    fn execute(self, context: &mut C) -> Self::Continue {
        // Receive a message
        let _message = T::from(context.receive().expect("Failed to receive"));
        // Continue with the next step
        self.next.execute(context)
    }
}

impl<C: Context> Protocol<C> for End {
    type Continue = ();
    
    fn execute(self, _context: &mut C) -> Self::Continue {
        // End of protocol
        ()
    }
}


### Ping Pong Example

In [ ]:

// Message types for our ping-pong protocol
enum PingMsg { Ping }
enum PongMsg { Pong }

impl Into<Vec<u8>> for PingMsg {
    fn into(self) -> Vec<u8> {
        b"PING".to_vec()
    }
}

impl Into<Vec<u8>> for PongMsg {
    fn into(self) -> Vec<u8> {
        b"PONG".to_vec()
    }
}

impl From<Vec<u8>> for PingMsg {
    fn from(_bytes: Vec<u8>) -> Self {
        PingMsg::Ping
    }
}

impl From<Vec<u8>> for PongMsg {
    fn from(_bytes: Vec<u8>) -> Self {
        PongMsg::Pong
    }
}

// Example usage - Client role (Ping sender)
fn ping_client() -> impl Protocol<TcpStream> {
    // μX.Send<Ping>.Receive<Pong>.X
    Mu(|rec| {
        Send {
            message: PingMsg::Ping,
            next: Receive {
                _phantom: PhantomData::<PongMsg>,
                next: rec,
            },
        }
    })
}

// Server role (Pong responder)
fn pong_server() -> impl Protocol<TcpStream> {
    // μX.Receive<Ping>.Send<Pong>.X
    Mu(|rec| {
        Receive {
            _phantom: PhantomData::<PingMsg>,
            next: Send {
                message: PongMsg::Pong,
                next: rec,
            },
        }
    })
}

// In actual use:
fn main() -> std::io::Result<()> {
    // Client example (pseudo-code)
    // let stream = TcpStream::connect("127.0.0.1:8080")?;
    // ping_client().execute(&mut stream);
    
    // Server example (pseudo-code)
    // let listener = TcpListener::bind("127.0.0.1:8080")?;
    // let (stream, _) = listener.accept()?;
    // pong_server().execute(&mut stream);
    
    Ok(())
}


// Approach 4: Session Type Combinators with Fixed Points

use std::marker::PhantomData;
use std::io::{Read, Write};

// Protocol trait with associated types for continuations
trait Protocol {
    // The type that the protocol continues to after execution
    type Next;
    
    // Run the protocol on the given channel
    fn run<C: Channel>(self, channel: C) -> (Self::Next, C);
}

// Channel abstraction
trait Channel: Read + Write {}

// Fixed-point operator for recursive protocols
struct Fix<F>(F);

// Protocol that can be unfolded
trait Unfold {
    type Unfolded: Protocol;
    fn unfold(self) -> Self::Unfolded;
}

// Implementation of unfolding for Fix
impl<F, P: Protocol> Unfold for Fix<F>
where
    F: FnOnce(Fix<F>) -> P,
{
    type Unfolded = P;
    
    fn unfold(self) -> P {
        (self.0)(self)
    }
}

// Implementation of Protocol for Fix via unfolding
impl<F, P: Protocol> Protocol for Fix<F>
where
    F: FnOnce(Fix<F>) -> P,
    Fix<F>: Unfold<Unfolded = P>,
{
    type Next = P::Next;
    
    fn run<C: Channel>(self, channel: C) -> (Self::Next, C) {
        self.unfold().run(channel)
    }
}

// Basic protocol combinators
struct Send<T, P> {
    message: T,
    next: P,
}

struct Receive<T, P> {
    _phantom: PhantomData<T>,
    next: P,
}

struct End;

// Protocol implementation for Send
impl<T: Into<Vec<u8>>, P: Protocol> Protocol for Send<T, P> {
    type Next = P::Next;
    
    fn run<C: Channel>(self, mut channel: C) -> (Self::Next, C) {
        let data: Vec<u8> = self.message.into();
        channel.write_all(&data).expect("Failed to send");
        self.next.run(channel)
    }
}

// Protocol implementation for Receive
impl<T: TryFrom<Vec<u8>>, P: Protocol> Protocol for Receive<T, P> {
    type Next = P::Next;
    
    fn run<C: Channel>(self, mut channel: C) -> (Self::Next, C) {
        let mut buffer = Vec::new();
        channel.read_to_end(&mut buffer).expect("Failed to receive");
        let _: T = T::try_from(buffer).expect("Failed to parse message");
        self.next.run(channel)
    }
}

// Protocol implementation for End
impl Protocol for End {
    type Next = ();
    
    fn run<C: Channel>(self, channel: C) -> (Self::Next, C) {
        ((), channel)
    }
}

// Message types
struct PingMessage;
struct PongMessage;

impl Into<Vec<u8>> for PingMessage {
    fn into(self) -> Vec<u8> {
        b"PING".to_vec()
    }
}

impl Into<Vec<u8>> for PongMessage {
    fn into(self) -> Vec<u8> {
        b"PONG".to_vec()
    }
}

impl TryFrom<Vec<u8>> for PingMessage {
    type Error = ();
    
    fn try_from(_bytes: Vec<u8>) -> Result<Self, Self::Error> {
        Ok(PingMessage)
    }
}

impl TryFrom<Vec<u8>> for PongMessage {
    type Error = ();
    
    fn try_from(_bytes: Vec<u8>) -> Result<Self, Self::Error> {
        Ok(PongMessage)
    }
}

// Simple fixed point type
type RecursiveProtocol<F> = Fix<F>;

// Implementation for a TcpStream as a Channel
impl Channel for TcpStream {}

// Example ping-pong protocol definitions
fn ping_client() -> impl Protocol<Next = ()> {
    // Create a recursive ping-pong protocol where we:
    // 1. Send a ping
    // 2. Receive a pong
    // 3. Recursively continue from step 1
    RecursiveProtocol(|rec| {
        Send {
            message: PingMessage,
            next: Receive {
                _phantom: PhantomData::<PongMessage>,
                next: rec,
            },
        }
    })
}

fn pong_server() -> impl Protocol<Next = ()> {
    // Create a recursive ping-pong protocol where we:
    // 1. Receive a ping
    // 2. Send a pong
    // 3. Recursively continue from step 1
    RecursiveProtocol(|rec| {
        Receive {
            _phantom: PhantomData::<PingMessage>,
            next: Send {
                message: PongMessage,
                next: rec,
            },
        }
    })
}

// Main function would connect and run the protocols
fn main() -> std::io::Result<()> {
    // This is just example code showing how you would use these protocols
    
    // Client example
    // let stream = TcpStream::connect("127.0.0.1:8080")?;
    // let (_, _) = ping_client().run(stream);
    
    // Server example
    // let listener = TcpListener::bind("127.0.0.1:8080")?;
    // let (stream, _) = listener.accept()?;
    // let (_, _) = pong_server().run(stream);
    
    Ok(())
}


// Extended Example: Adding Choice to the Ping-Pong Protocol
// This demonstrates how to model choices in recursive protocols

// First, let's define sum types for choices
enum Either<L, R> {
    Left(L),
    Right(R),
}

// Protocol combinator for choices
struct Choose<L, R> {
    choice: Either<L, R>,
}

// Protocol combinator for offering choices
struct Offer<L, R> {
    left: L,
    right: R,
}

// Implementation for Choose
impl<L: Protocol, R: Protocol<Next = L::Next>> Protocol for Choose<L, R> {
    type Next = L::Next;
    
    fn run<C: Channel>(self, channel: C) -> (Self::Next, C) {
        match self.choice {
            Either::Left(l) => {
                // Signal we're taking the left branch
                let mut c = channel;
                c.write_all(&[0]).expect("Failed to send choice");
                l.run(c)
            },
            Either::Right(r) => {
                // Signal we're taking the right branch
                let mut c = channel;
                c.write_all(&[1]).expect("Failed to send choice");
                r.run(c)
            }
        }
    }
}

// Implementation for Offer
impl<L: Protocol, R: Protocol<Next = L::Next>> Protocol for Offer<L, R> {
    type Next = L::Next;
    
    fn run<C: Channel>(self, mut channel: C) -> (Self::Next, C) {
        // Read which branch was chosen
        let mut buffer = [0; 1];
        channel.read_exact(&mut buffer).expect("Failed to read choice");
        
        match buffer[0] {
            0 => self.left.run(channel),
            _ => self.right.run(channel),
        }
    }
}

// Now we can define a ping-pong protocol with the option to stop
fn ping_client_with_stop() -> impl Protocol<Next = ()> {
    RecursiveProtocol(|rec| {
        // Choose between continuing or stopping
        Choose {
            choice: Either::Left(
                // Continue with ping-pong
                Send {
                    message: PingMessage,
                    next: Receive {
                        _phantom: PhantomData::<PongMessage>,
                        next: rec,
                    },
                }
            ),
        }
    })
}

fn pong_server_with_stop() -> impl Protocol<Next = ()> {
    RecursiveProtocol(|rec| {
        // Offer the client to continue or stop
        Offer {
            // Client wants to continue
            left: Receive {
                _phantom: PhantomData::<PingMessage>,
                next: Send {
                    message: PongMessage,
                    next: rec,
                },
            },
            // Client wants to stop
            right: End,
        }
    })
}